Reading data from file

In [36]:
import csv
import json
from nltk.tokenize import TweetTokenizer

def read_hate_tweets (annofile, jsonfile):
    """Reads in hate speech data."""
    all_data = {}
    annos = {}
    with open(annofile) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        for row in csvreader:
            if row[0] in annos:
                # if duplicate with different rating, remove!
                if row[1] != annos[row[0]]:
                    del(annos[row[0]])
            else:
                annos[row[0]] = row[1]

    tknzr = TweetTokenizer()
                
    with open(jsonfile) as jsonfile:
        for line in jsonfile:
            twtjson = json.loads(line)
            twt_id = twtjson['id_str']
            if twt_id in annos:
                all_data[twt_id] = {}
                all_data[twt_id]['offensive'] = "nonoffensive" if annos[twt_id] == 'none' else "offensive"
                all_data[twt_id]['text_tok'] = tknzr.tokenize(twtjson['text'])

    # split training and test data:
    all_data_sorted = sorted(all_data.items())
    items = [(i[1]['text_tok'],i[1]['offensive']) for i in all_data_sorted]
    splititem = len(all_data)-3250
    train_dt = items[:splititem]
    test_dt = items[splititem:]
    print('Training data:',len(train_dt))
    print('Test data:',len(test_dt))

    return(train_dt,test_dt)

TWEETS_ANNO = '../Data/NAACL_SRW_2016.csv'
TWEETS_TEXT = '../Data/NAACL_SRW_2016_tweets.json'

(train_data,test_data) = read_hate_tweets(TWEETS_ANNO,TWEETS_TEXT)


Training data: 12896
Test data: 3250


In [37]:
import numpy as np

def build_w2i(data):
    '''
    This function creates a vector of unique words in dataset excluding some stop words.
    
    '''
    vocab = []
    for word in data:
        vocab+=word[0]
    return set(vocab)
def featurize(data):
    
    '''
    This function creates a matrix (X) with rows representing each data instance and columns representing features.
    Features in this case is Vocabulary set. It assigns value 1 if word is present in data instance (tweet) else
    it assigns 0. 
    
    It creates another matrix (Y) with rows representing number of data instances and column represented by number of classes.
    
    '''
    
    X = np.array([[0 for j in range(len(vocab))],]*len(data))
    i=0
    for obj in data:
        tweet = [word.lower() for word in list(obj[0])]
        j=0
        for word in vocab:
            if(word in tweet):
                X[i][j] = 1
            else:
                X[i][j] = 0
            j+=1
        i+=1  
    Y = np.array([[0,1],]*len(data))
    for i in range(len(data)):
        if(data[i][1]=='offensive'):
            Y[i] = [1,0]
    return (X,Y)


In [38]:
vocab = list(build_w2i(train_data))
(x,y) = featurize(train_data)


In [93]:
import numpy as np

class LogReg:

    num_class = 0
    def __init__(self, eta=0.01, num_iter=30):
        self.eta = eta
        self.num_iter = num_iter
        
    def softmax(self,x):
        '''
        This function computes softmax value for an array or a matrix

        Input : x is a array or a matrix

        Output : result is either a array of softmax value or a matrix
        '''
        x=x.astype(float)
        if x.ndim==1:
            return np.exp(x)/np.sum(np.exp(x))
        elif x.ndim==2:
            result=np.zeros_like(x)
            M,N=x.shape
            z = x - np.max(x, axis=0, keepdims=True)
            for n in range(N):
                S=np.sum(np.exp(z[:,n]))
                result[:,n]=np.exp(z[:,n])/S
            return result
        else:
            print("The input array is not 1- or 2-dimensional.")
    
    
    def gradient(self,y_pred,Y,X):
        
        '''
        This function computes gradient using calculated output (y_), actual output(y) and input matrix (x)
        x is matrix which contains data instance wrt their weights.
        
        "grad" variable is a matrix with rows representing number of classes (2) and column representing 
         features(words in Vocabulary).
        
        It returns improved weights and bias for each features wrt their classes.
        
        '''
        temp = np.subtract(y_pred,Y)
        weight = np.matmul(temp,X)/len(X)
        bias = np.matmul(temp,np.ones((X.shape)))/len(X)
        return (weight,bias)
    
    def cost(self,y_pred,Y):
        error = np.sum(-np.multiply(Y,np.log(y_pred,where= y_pred !=0)))
        return error
        
    
    
    def train(self, X, Y):
        '''
        This function trains given dataset by iteratively updating their weights and bias.
        weights is matrix of size (num of classes X num of words in Vocabulary)
        bias is matrix of size (num of classes X 1)
        
        We divide dataset into mini-batches of 100 data instances and update weights and bias at end of each mini batch.
        We repeat this process num_iter times
        
        '''
        # weights initialization
        self.num_class = Y.shape[1]
        self.weights = np.zeros((Y.shape[1],X.shape[1]))
        self.bias = np.zeros((Y.shape[1],X.shape[1]))
        for i in range(self.num_iter):
            ind = np.arange(len(X))
            np.random.shuffle(ind)
            b=np.arange(0,len(X),100)
            np.append(b,len(X))
            loss = 0
            j=0
            for j in range(len(b)-1):
                x = X[ind[b[j]:b[j+1]]]
                y = Y[ind[b[j]:b[j+1]]]
                prob = np.zeros((2,len(x)))
                prob = self.p(x)
                error = self.cost(prob,y.T)
                loss = loss + error/len(x)
                change_in_w, change_in_b = self.gradient(prob,y.T,x)
                self.weights = self.weights - self.eta*change_in_w
                self.bias = self.bias - self.eta*change_in_b
                
            print("Loss : ",loss)
        return None
    
    
    def p(self, X): 
        '''
        This function cpmputes probability for each data instance wrt each class.
        
        "prob" is a matrix of size ( num of data instances X num of classes )
        For our particular case first case represent class "nonoffensive" and second column represent "offensive"
        i.e prob[i][0] is probability for y=0 and prob[i][1] is probability for y=1 where y is actual output and
        offensive means 1.
        
        '''
        temp = np.add(np.matmul(self.weights,X.T),np.matmul(self.bias,np.ones((X.T.shape))))
        prob = np.zeros((self.weights.shape))
        prob = self.softmax(temp) 
        return prob
    
    def predict(self, X):
        '''
        This function predicts the class to which given data instance belongs to.
        We compare the calculated probability of given instance among all classes.
        Class with max probability is the output.
        
        It also calculates F_1 score.
        '''
        (x_test,y_test) = featurize(X)
        prob=self.p(x_test)
        result=np.argmax(prob,axis=0)
        tp=0
        tn=0
        fn=0
        fp=0
        y=y_test.T
        print(y.shape)
        print(y_test.shape)
        for i in range(len(y_test)):
            if(y[0][i]==result[i]):
                if(result[i]==1):
                    tp+=1
                else:
                    tn+=1
            else:
                if(result[i]==1):
                    fp+=1
                else:
                    fn+=1
        pr = tp/(tp+fp)
        r = tp/(tp+fn)
        f1 = 2*pr*r/(pr+r)
        acc = (tp+tn)/(tp+tn+fn+fp)
        print("F_1 score is: "+str(f1))
        
        return None

In [94]:
lg=LogReg(0.2,30)
lg.train(x[:1000],y[:1000])
lg.predict(test_data[:100])

Loss :  4158.56323718056
Loss :  2917.643189999999
Loss :  1864.8614599999994
Loss :  2127.621210000002
Loss :  2823.9683599999926
Loss :  513.2477199999928
Loss :  3060.6792315205776
Loss :  3972.5052738831682
Loss :  5335.743563747172
Loss :  1960.6762014031058
Loss :  5167.948330175801
Loss :  337.3188105009033
Loss :  2948.4846683177734
Loss :  2778.7250643881257
Loss :  1383.7559333358859
Loss :  1949.2901446115234
Loss :  879.3674765007928
Loss :  2941.404088271625
Loss :  5198.309083337486
Loss :  3877.834442125842
Loss :  3109.841387686477
Loss :  10.219883019361419
Loss :  611.4186316294314
Loss :  3652.4036804040834
Loss :  2478.0501491471427
Loss :  3683.275104726764
Loss :  1777.9852560220588
Loss :  1807.3878909404177
Loss :  3701.4135159326006
Loss :  4881.4282416555
(2, 100)
(100, 2)
F_1 score is: 0.33333333333333337


In [90]:
lg.predict(test_data[:100])

ZeroDivisionError: division by zero